# Webcrawler for soccer data from http://www.kicker.de


In [2]:
# imports
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup

In [3]:
from googletrans import Translator

# init and test translator
translator = Translator()
translation = translator.translate("Der Himmel ist blau und ich mag Bananen", dest='en')

# sanity check
print(translation.text)

The sky is blue and I like bananas


In [4]:
# set base url
base_url = "https://www.kicker.de"

<div style='background:#75CF63; padding: 15px; border: thin solid black; border-radius: 5px; text-align: center'>
start getting all match urls for the relevant seasons



In [6]:
# go to the website of the current season and get all the html content
url_20_21 = "https://www.kicker.de/bundesliga/spieltag/2020-21/-1"
soup = BeautifulSoup(requests.get(url_20_21).text, 'html.parser')

In [7]:
# then grab all the dropdown options. 
dropdowns = soup.find_all("option")

# get all the links for all the options in the drop down. The first ~40 options are the links to seasons in descending order 
all_values = [ a['value'] for a in dropdowns ]

# grab the urls for the last 6 finished seasons (i.e. 2015/16 - 2019/20)
selected_season_urls = all_values[1:7]

print("selected seasons:", selected_season_urls)

selected seasons: ['/bundesliga/spieltag/2019-20', '/bundesliga/spieltag/2018-19', '/bundesliga/spieltag/2017-18', '/bundesliga/spieltag/2016-17', '/bundesliga/spieltag/2015-16', '/bundesliga/spieltag/2014-15']


In [70]:
all_links_per_season = {}

# loop over seasons
for season_url in selected_season_urls:
    
    # create a season string
    season = season_url.split('/')[3]    
    
    # create tmp soupfor each season
    season_soup = BeautifulSoup(requests.get(base_url+season_url+"/-1").text, 'html.parser')
    links_with_trash = season_soup.find_all("a", class_ = "kick__v100-scoreBoard kick__v100-scoreBoard--standard")
    all_paths = [ a['href'] for a in links_with_trash ]
    all_full_paths = [base_url + url for url in all_paths]
    
    # populate helper structure
    all_links_per_season[season] = all_full_paths
    
    # report
    print(f'all match links from season {season} stored')

all match links from season 2019-20 stored
all match links from season 2018-19 stored
all match links from season 2017-18 stored
all match links from season 2016-17 stored
all match links from season 2015-16 stored
all match links from season 2014-15 stored


<div style="background: #dadaeb; border: thin solid black; padding: 15px; margin: 15px">

prepare parsing functions

In [76]:
def get_home_stats(page_soup):
    game_info = page_soup.find_all("div", class_="kick__stats-bar__value kick__stats-bar__value--opponent1")

    data = []
    for info in game_info:
        
        info_text = re.sub("[A-z]", "", info.get_text())        
        info = info_text.replace("%", "").strip()
        info_final = info.replace(",", ".") 
        
        data.append(info_final)

    if len(data) < 5:
        return 'error'
    else:
        return {"shots": data[0], "distance": data[1], "passes": data[2], "misplaced_passes": data[3], "pass_accuracy": data[4], 'possession': data[5]}

def get_away_stats(page_soup):
    game_info = page_soup.find_all("div", class_="kick__stats-bar__value kick__stats-bar__value--opponent2")

    data = []
    for info in game_info:
        
        info_text = re.sub("[A-z]", "", info.get_text())        
        info = info_text.replace("%", "").strip()
        info_final = info.replace(",", ".") 
        
        data.append(info_final)
    
    if len(data) < 5:
        return 'error'
    else:
        return {"shots": data[0], "distance": data[1], "passes": data[2], "misplaced_passes": data[3], "pass_accuracy": data[4], 'possession': data[5]}


In [146]:

def get_match_info(match_soup):

    # grab date
    game_info = match_soup.find_all("div", class_="kick__gameinfo__item kick__gameinfo__item--game-preview")
    p_tags = game_info[0].find_all('p')
    date_string = re.sub("[A-zß]", "", p_tags[0].get_text() )
    
     # grab home & away team
    team_home = match_soup.find_all("div", class_="kick__v100-gameCell__team__name")[0].string.strip()
    team_away = match_soup.find_all("div", class_="kick__v100-gameCell__team__name")[1].string.strip()
    
    # grab scores
    score_home = match_soup.find_all("div", class_="kick__v100-scoreBoard__scoreHolder__score")[0].string.strip()
    score_away = match_soup.find_all("div", class_="kick__v100-scoreBoard__scoreHolder__score")[1].string.strip()
    
    # get detailed stats for home and away teams 
    home_stats = get_home_stats(match_soup)
    away_stats = get_away_stats(match_soup)

    # get shots
    shots_home = home_stats['shots']
    shots_away = away_stats['shots']

    # get passes
    passes_home = home_stats['passes']
    passes_away = away_stats['passes']
    
    # get passes
    misplaced_passes_home = home_stats['misplaced_passes']
    misplaced_passes_away = away_stats['misplaced_passes'] 
    
    # get passes
    pass_accuracy_home = home_stats['pass_accuracy']
    pass_accuracy_away = away_stats['pass_accuracy']
    
    # get passes
    distance_home = home_stats['distance']
    distance_away = away_stats['distance']
    
    # get possession
    possession_home = home_stats['possession']
    possession_away = away_stats['possession']
    
    # get grade
    grade_div = match_soup.find_all("div", class_="kick__badge-xbig")
    grade = grade_div[0].string
    
    # grab the match grade summary
    grade_summary = match_soup.find_all("p", class_="kick__gameinfo__long-text") 
    summary_german = grade_summary[0].string  
    #summary_english = 'TODO = need to pay for google translate API (just a few cents, so no prob, but we would like to get the OK for this project first)'
    
    summary_english = translator.translate(summary_german, src='de', dest='en').text

    
    return {
        "date": date_string,
        "name_home_team": team_home,
        "name_away_team": team_away,
        "score_home": score_home,
        "score_away": score_away,
        "shots_home": shots_home,
        "shots_away": shots_away,
        "passes_home": passes_home,
        "passes_away": passes_away,
        "misplaced_passes_home": misplaced_passes_home,
        "misplaced_passes_away": misplaced_passes_away,
        "pass_accuracy_home": pass_accuracy_home,
        "pass_accuracy_away": pass_accuracy_away,
        "distance_home": distance_home,
        "distance_away": distance_away,
        "grade": grade,
        "summary_german": summary_german,
        "summary_english": summary_english,
        "coach_home": 'TODO',
        "coach_away": 'TODO',
        "interview_german": 'TODO',
        "interview_english": 'TODO'
    }

#home_team, away_team, date_string, grade, summary, home_stats]

In [156]:
def reconstruct_name(kicker_name_string):
    name_components = re.sub( r"([A-Z])", r" \1", kicker_name_string).split()

    return name_components[1] + ' ' + name_components[0]


def get_coach_names(soup):
    #coach = soup.find_all("div", class_="kick__data-list__row kick__data-list__row--image")#.string.strip()
    coach_home = reconstruct_name(soup.find_all("a", class_="kick__data-list__value")[0].text)
    coach_away = reconstruct_name(soup.find_all("a", class_="kick__data-list__value")[1].text)

    return coach_home, coach_away


<div style="background: #dadaeb; border: thin solid black; padding: 15px; margin: 15px">

Prepare final data frame to store all info

In [157]:
# get info for random match to get all the keys so that we can use those for our col names

In [149]:
sample_match_url = 'https://www.kicker.de/bayern-gegen-schalke-2020-bundesliga-4667048/analyse'

sample_match_soup = BeautifulSoup(requests.get(sample_match_url.replace('analyse', 'spielinfo')).text, 'html.parser')    
sample_match_info = get_match_info(sample_match_soup)

# column names
column_names = sample_match_info.keys()

# init final df
df_final_structure = pd.DataFrame(columns=column_names)

<div style="background: #dadaeb; border: thin solid black; padding: 15px; margin: 15px">

scrape all matches in all seasons

In [161]:
for season in all_links_per_season:
    
    tmp_df = df_final_structure.copy()
    
    all_match_urls_in_season = all_links_per_season[season]
    
    for index, match_url in enumerate(all_match_urls_in_season):
        print(f'scraping {match_url}')
        
        # get beautiful soups for game info and line-up pages
        match_info_soup = BeautifulSoup(requests.get(match_url.replace('analyse', 'spielinfo')).text, 'html.parser')
        line_up_info_soup = BeautifulSoup(requests.get(match_url.replace('analyse', 'aufstellung')).text, 'html.parser')
        
        # get match info from match info page
        match_info = get_match_info(match_info_soup)
        
        # get coach names from line-up page
        coaches_names = get_coach_names(line_up_info_soup)
        
        # add coach info to match_info dictionary which will serve as the complete info data structure
        match_info['coach_home'] = coaches_names[0]
        match_info['coach_away'] = coaches_names[1]

        
        # if get_match_info didn't return an error, append match info to tmp_df
        if match_info != "error":
            tmp_df.loc[index] = list(match_info.values())
        else:
            print(f'error at {season} with url {match_url}')
        
        if index%10 == 0:
            print(f'quick sleep at index: {index}, current season: {season}')
            sleep(0.5)

    # save scraped info as csv (by season)        
    tmp_df.to_csv('data/' + season+'__match_infos_with_grades_and_summary.csv', index = False, encoding='utf-8-sig')

scraping https://www.kicker.de/bayern-gegen-hertha-2019-bundesliga-4588640/analyse
quick sleep at index: 0, current season: 2019-20
scraping https://www.kicker.de/dortmund-gegen-augsburg-2019-bundesliga-4588642/analyse
scraping https://www.kicker.de/leverkusen-gegen-paderborn-2019-bundesliga-4588643/analyse
scraping https://www.kicker.de/wolfsburg-gegen-koeln-2019-bundesliga-4588646/analyse
scraping https://www.kicker.de/bremen-gegen-duesseldorf-2019-bundesliga-4588649/analyse
scraping https://www.kicker.de/freiburg-gegen-mainz-2019-bundesliga-4588650/analyse
scraping https://www.kicker.de/gladbach-gegen-schalke-2019-bundesliga-4588644/analyse
scraping https://www.kicker.de/frankfurt-gegen-hoffenheim-2019-bundesliga-4588647/analyse
scraping https://www.kicker.de/union-gegen-leipzig-2019-bundesliga-4588651/analyse
scraping https://www.kicker.de/koeln-gegen-dortmund-2019-bundesliga-4588665/analyse
scraping https://www.kicker.de/hoffenheim-gegen-bremen-2019-bundesliga-4588658/analyse
quic

scraping https://www.kicker.de/koeln-gegen-hoffenheim-2019-bundesliga-4588755/analyse
quick sleep at index: 90, current season: 2019-20
scraping https://www.kicker.de/hertha-gegen-leipzig-2019-bundesliga-4588751/analyse
scraping https://www.kicker.de/mainz-gegen-union-2019-bundesliga-4588752/analyse
scraping https://www.kicker.de/schalke-gegen-duesseldorf-2019-bundesliga-4588754/analyse
scraping https://www.kicker.de/paderborn-gegen-augsburg-2019-bundesliga-4588756/analyse
scraping https://www.kicker.de/bayern-gegen-dortmund-2019-bundesliga-4588748/analyse
scraping https://www.kicker.de/gladbach-gegen-bremen-2019-bundesliga-4588749/analyse
scraping https://www.kicker.de/wolfsburg-gegen-leverkusen-2019-bundesliga-4588750/analyse
scraping https://www.kicker.de/freiburg-gegen-frankfurt-2019-bundesliga-4588753/analyse
scraping https://www.kicker.de/dortmund-gegen-paderborn-2019-bundesliga-4588757/analyse
scraping https://www.kicker.de/leverkusen-gegen-freiburg-2019-bundesliga-4588759/analy

scraping https://www.kicker.de/frankfurt-gegen-augsburg-2020-bundesliga-4588842/analyse
quick sleep at index: 180, current season: 2019-20
scraping https://www.kicker.de/wolfsburg-gegen-duesseldorf-2020-bundesliga-4588841/analyse
scraping https://www.kicker.de/bremen-gegen-union-2020-bundesliga-4588843/analyse
scraping https://www.kicker.de/hertha-gegen-mainz-2020-bundesliga-4588844/analyse
scraping https://www.kicker.de/freiburg-gegen-hoffenheim-2020-bundesliga-4588845/analyse
scraping https://www.kicker.de/schalke-gegen-paderborn-2020-bundesliga-4588846/analyse
scraping https://www.kicker.de/leverkusen-gegen-dortmund-2020-bundesliga-4588839/analyse
scraping https://www.kicker.de/bayern-gegen-leipzig-2020-bundesliga-4588838/analyse
scraping https://www.kicker.de/gladbach-gegen-koeln-2020-bundesliga-4588840/analyse
scraping https://www.kicker.de/dortmund-gegen-frankfurt-2020-bundesliga-4588847/analyse
scraping https://www.kicker.de/leipzig-gegen-bremen-2020-bundesliga-4588848/analyse
q

scraping https://www.kicker.de/hoffenheim-gegen-leipzig-2020-bundesliga-4588930/analyse
quick sleep at index: 270, current season: 2019-20
scraping https://www.kicker.de/wolfsburg-gegen-freiburg-2020-bundesliga-4588929/analyse
scraping https://www.kicker.de/duesseldorf-gegen-dortmund-2020-bundesliga-4588931/analyse
scraping https://www.kicker.de/hertha-gegen-frankfurt-2020-bundesliga-4588932/analyse
scraping https://www.kicker.de/koeln-gegen-union-2020-bundesliga-4588935/analyse
scraping https://www.kicker.de/paderborn-gegen-bremen-2020-bundesliga-4588936/analyse
scraping https://www.kicker.de/bayern-gegen-gladbach-2020-bundesliga-4588928/analyse
scraping https://www.kicker.de/mainz-gegen-augsburg-2020-bundesliga-4588933/analyse
scraping https://www.kicker.de/schalke-gegen-leverkusen-2020-bundesliga-4588934/analyse
scraping https://www.kicker.de/gladbach-gegen-wolfsburg-2020-bundesliga-4588940/analyse
scraping https://www.kicker.de/bremen-gegen-bayern-2020-bundesliga-4588942/analyse
qu

scraping https://www.kicker.de/augsburg-gegen-freiburg-2018-bundesliga-4243316/analyse
scraping https://www.kicker.de/bremen-gegen-wolfsburg-2018-bundesliga-4243323/analyse
scraping https://www.kicker.de/dortmund-gegen-augsburg-2018-bundesliga-4243321/analyse
scraping https://www.kicker.de/hannover-gegen-stuttgart-2018-bundesliga-4243324/analyse
scraping https://www.kicker.de/mainz-gegen-hertha-2018-bundesliga-4243326/analyse
scraping https://www.kicker.de/duesseldorf-gegen-schalke-2018-bundesliga-4243327/analyse
scraping https://www.kicker.de/bayern-gegen-gladbach-2018-bundesliga-4243319/analyse
scraping https://www.kicker.de/freiburg-gegen-leverkusen-2018-bundesliga-4243325/analyse
quick sleep at index: 60, current season: 2018-19
scraping https://www.kicker.de/hoffenheim-gegen-frankfurt-2018-bundesliga-4243320/analyse
scraping https://www.kicker.de/leipzig-gegen-nuernberg-2018-bundesliga-4243322/analyse
scraping https://www.kicker.de/frankfurt-gegen-duesseldorf-2018-bundesliga-42433

...

scraping https://www.kicker.de/augsburg-gegen-leverkusen-2016-bundesliga-2855397/analyse
scraping https://www.kicker.de/bremen-gegen-hannover-2016-bundesliga-2855398/analyse
scraping https://www.kicker.de/frankfurt-gegen-ingolstadt-2016-bundesliga-2855399/analyse
quick sleep at index: 220, current season: 2015-16
scraping https://www.kicker.de/koeln-gegen-schalke-2016-bundesliga-2855400/analyse
scraping https://www.kicker.de/dortmund-gegen-bayern-2016-bundesliga-2855393/analyse
scraping https://www.kicker.de/mainz-gegen-darmstadt-2016-bundesliga-2855401/analyse
scraping https://www.kicker.de/hsv-gegen-hertha-2016-bundesliga-2855395/analyse
scraping https://www.kicker.de/hertha-gegen-schalke-2016-bundesliga-2855407/analyse
scraping https://www.kicker.de/gladbach-gegen-frankfurt-2016-bundesliga-2855405/analyse
scraping https://www.kicker.de/hannover-gegen-koeln-2016-bundesliga-2855406/analyse
scraping https://www.kicker.de/hoffenheim-gegen-wolfsburg-2016-bundesliga-2855408/analyse
scrapi

scraping https://www.kicker.de/hannover-gegen-schalke-2014-bundesliga-2407222/analyse
scraping https://www.kicker.de/hertha-gegen-bremen-2014-bundesliga-2407223/analyse
scraping https://www.kicker.de/frankfurt-gegen-freiburg-2014-bundesliga-2407224/analyse
scraping https://www.kicker.de/koeln-gegen-hsv-2014-bundesliga-2407225/analyse
scraping https://www.kicker.de/dortmund-gegen-leverkusen-2014-bundesliga-2407219/analyse
scraping https://www.kicker.de/paderborn-gegen-mainz-2014-bundesliga-2407226/analyse
scraping https://www.kicker.de/gladbach-gegen-stuttgart-2014-bundesliga-2407220/analyse
scraping https://www.kicker.de/augsburg-gegen-dortmund-2014-bundesliga-2407240/analyse
scraping https://www.kicker.de/leverkusen-gegen-hertha-2014-bundesliga-2407237/analyse
quick sleep at index: 10, current season: 2014-15
scraping https://www.kicker.de/wolfsburg-gegen-frankfurt-2014-bundesliga-2407238/analyse
scraping https://www.kicker.de/bremen-gegen-hoffenheim-2014-bundesliga-2407241/analyse
sc

scraping https://www.kicker.de/hoffenheim-gegen-koeln-2014-bundesliga-2407321/analyse
scraping https://www.kicker.de/frankfurt-gegen-bayern-2014-bundesliga-2407324/analyse
scraping https://www.kicker.de/freiburg-gegen-schalke-2014-bundesliga-2407325/analyse
scraping https://www.kicker.de/bremen-gegen-stuttgart-2014-bundesliga-2407323/analyse
scraping https://www.kicker.de/wolfsburg-gegen-hsv-2014-bundesliga-2407319/analyse
scraping https://www.kicker.de/dortmund-gegen-gladbach-2014-bundesliga-2407317/analyse
scraping https://www.kicker.de/bayern-gegen-hoffenheim-2014-bundesliga-2407326/analyse
scraping https://www.kicker.de/schalke-gegen-wolfsburg-2014-bundesliga-2407327/analyse
quick sleep at index: 100, current season: 2014-15
scraping https://www.kicker.de/gladbach-gegen-frankfurt-2014-bundesliga-2407328/analyse
scraping https://www.kicker.de/mainz-gegen-freiburg-2014-bundesliga-2407329/analyse
scraping https://www.kicker.de/hannover-gegen-leverkusen-2014-bundesliga-2407330/analyse


scraping https://www.kicker.de/hoffenheim-gegen-stuttgart-2015-bundesliga-2407411/analyse
scraping https://www.kicker.de/bremen-gegen-augsburg-2015-bundesliga-2407414/analyse
scraping https://www.kicker.de/frankfurt-gegen-schalke-2015-bundesliga-2407415/analyse
scraping https://www.kicker.de/hertha-gegen-freiburg-2015-bundesliga-2407413/analyse
scraping https://www.kicker.de/hannover-gegen-paderborn-2015-bundesliga-2407412/analyse
scraping https://www.kicker.de/stuttgart-gegen-dortmund-2015-bundesliga-2407421/analyse
scraping https://www.kicker.de/schalke-gegen-bremen-2015-bundesliga-2407416/analyse
quick sleep at index: 190, current season: 2014-15
scraping https://www.kicker.de/mainz-gegen-frankfurt-2015-bundesliga-2407418/analyse
scraping https://www.kicker.de/augsburg-gegen-leverkusen-2015-bundesliga-2407419/analyse
scraping https://www.kicker.de/freiburg-gegen-hoffenheim-2015-bundesliga-2407420/analyse
scraping https://www.kicker.de/paderborn-gegen-bayern-2015-bundesliga-2407424/a

scraping https://www.kicker.de/bremen-gegen-frankfurt-2015-bundesliga-2407504/analyse
scraping https://www.kicker.de/freiburg-gegen-paderborn-2015-bundesliga-2407505/analyse
scraping https://www.kicker.de/leverkusen-gegen-bayern-2015-bundesliga-2407498/analyse
scraping https://www.kicker.de/mainz-gegen-hsv-2015-bundesliga-2407500/analyse
scraping https://www.kicker.de/hertha-gegen-gladbach-2015-bundesliga-2407503/analyse
scraping https://www.kicker.de/hsv-gegen-freiburg-2015-bundesliga-2407512/analyse
scraping https://www.kicker.de/bayern-gegen-augsburg-2015-bundesliga-2407506/analyse
quick sleep at index: 280, current season: 2014-15
scraping https://www.kicker.de/dortmund-gegen-hertha-2015-bundesliga-2407507/analyse
scraping https://www.kicker.de/gladbach-gegen-leverkusen-2015-bundesliga-2407508/analyse
scraping https://www.kicker.de/hannover-gegen-bremen-2015-bundesliga-2407509/analyse
scraping https://www.kicker.de/frankfurt-gegen-hoffenheim-2015-bundesliga-2407510/analyse
scraping